In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import ndpretty

import data_prep
import gnn_prep
import widget_util

In [2]:
ndpretty.default()
data = data_prep.load_beijing_data()


Loaded air quality data from 87 devices. No weather data for []


# Analyse experiment results

In [3]:
gnn_data_path="Previous work/PM2.5-GNN/data/"
gnn_experiment_path = gnn_data_path + "results/"

dataset_name = "DS2"

devices_path = f"{gnn_data_path}city_{dataset_name}.txt"
city_txt = pd.read_csv(devices_path, sep=' ', header=None, index_col=0)
devices = list(city_txt[1])

In [4]:
class Experiment:
    def __init__(self, name, path, start_date, devices):
        self.name = name
        self.path = path
        self.start_date = start_date
        self.devices = devices
        self.preds, self.R_dict = self.load_experiment()

    def load_experiment(self):
        time_npy = np.load(self.path + 'time.npy')
        pred_npy = np.load(self.path + 'predict.npy')

        preds = gnn_prep.transform_preds_back(time_npy, pred_npy, self.devices, self.start_date, melted=True)

        try:
            R_npy = np.load(self.path + 'R.npy')
            R_dict = gnn_prep.transform_R_back(time_npy, R_npy, self.devices, self.start_date, self.name)
        except FileNotFoundError as e:
            print(f"Couldn't load R.npy for '{self.name}': {e}")
            R_dict = None

        preds['time'] = preds.index
        return preds, R_dict

    def __repr__(self):
        return f"Experiment '{self.name}'"
        

In [7]:
experiments = {
    "PM25_GNN": "0_336/1/PM25_GNN/2021-03-29_09-16-24/01/",
    "SGNN3_00": "0_336/1/SplitGNN_3/2021-03-29_20-33-32/00/",
    "SGNN3_01": "0_336/1/SplitGNN_3/2021-03-29_20-33-32/01/",
    "SGNN3_02": "0_336/1/SplitGNN_3/2021-03-29_20-33-32/02/",
    "SGNN3_03": "0_336/1/SplitGNN_3/2021-03-29_20-33-32/03/",
    "SGNN3_04": "0_336/1/SplitGNN_3/2021-03-29_20-33-32/04/",
}

start_date = "2020-01-01 01:00:00"
end_date = "2020-03-01 01:00:00"
for name, path in tqdm(experiments.items()):
    experiments[name] = Experiment(name, gnn_experiment_path + path, start_date, devices)

print(f"Loaded {len(experiments)} experiments")


Loaded 6 experiments


In [8]:
df = data.measurements.copy()
df.index = df['time']
ground_truth = df[df['device_id'].isin(devices)]

In [9]:
device_widget = widgets.Select(
    options=devices,
    rows=5,
    description='Device'
)

exp_widget = widgets.SelectMultiple(
    options=experiments.keys(),
    rows=10,
    description='Exps'
)

date_widget = widget_util.get_date_slider(data, start=start_date)

In [10]:
def plot_results(ground_truth, exps, selected_exps, device, date_range, feature, events):
    plt.figure(figsize=(10,5))

    # plot events
    for event in events:
        plt.axvspan(event.start, event.end, color=event.color, alpha=0.1, label=event.name)

    # plot ground truth
    (start, end) = date_range
    y = ground_truth[(ground_truth['device_id'] == device) & (ground_truth['time'] >= start) & (ground_truth['time'] < end)]
    plt.plot(y['time'], y[feature], label='ground truth')

    # plot experiment predictions
    for e in selected_exps:
        exp = exps[e]
        df = exp.preds
        y = df[(df['device_id'] == device) & (df['time'] >= start) & (df['time'] < end)]
        plt.plot(y['time'], y[feature], label=e)

    plt.tight_layout()
    plt.title(f"{feature} @ {device}")
    plt.legend(loc='upper left')
    axes = plt.gca()
    axes.set_xlim([start,end])
    plt.show()

    # plot R values for device
    for e in selected_exps:
        R_dict = exps[e].R_dict
        if R_dict is not None:
            R_dict[device].plot(kind='area', title=f"R for {e} @ {device}")

interact_manual(plot_results, ground_truth=fixed(ground_truth), exps=fixed(experiments), selected_exps=exp_widget, device=device_widget, date_range=date_widget, feature=fixed('pm25'), events=fixed(data.events))

interactive(children=(SelectMultiple(description='Exps', options=('PM25_GNN', 'SGNN3_00', 'SGNN3_01', 'SGNN3_0…

<function __main__.plot_results(ground_truth, exps, selected_exps, device, date_range, feature, events)>